In [1]:
import os
import glob
import pickle
import itertools
import numpy as np
from pathlib import Path
import collections
from matplotlib import pyplot as plt
from plotting import plot_est_and_gt_ellipses_on_images, plot_3D_scene
from lfd import compute_estimates, dual_quadric_to_ellipsoid_parameters

In [20]:
# Utilities
def read_list_poses(list):
    for idx, file_path in enumerate(list):
        with open(file_path) as f_input:
            pose = np.transpose(np.loadtxt(f_input)[:3, :])
            if idx == 0:
                poses = pose
            else:
                poses = np.concatenate((poses, pose), axis=0)
    return poses


def read_list_box(list):
    corpus = []
    for file_path in list:
        with open(file_path) as f_input:
            line = f_input.read()
            corpus.append([int(number) for number in line.split(",")])
    return np.array(corpus)

In [30]:
def sort_poses(pose_list):
    files = {int(Path(file).stem) : file for file in pose_list}
    ordered_dict = collections.OrderedDict(sorted(files.items()))
    return ordered_dict.values()


Load the input data.
Data association is implicitly defined in the data structures: each column of "visibility" corresponds to one object.
The information in "bbs" is structured in a similar way, with four columns for each object.

In [31]:
#dataset = "Aldoma"
dataset = "tiger"

save_output_images = True
random_downsample = False

if dataset != "Aldoma":
    PATH = f"data/{dataset}"
    box_list = glob.glob(os.path.join(os.getcwd(), f"{PATH}/bounding_boxes", "*.txt"))
    raw_poses_list = glob.glob(os.path.join(os.getcwd(), f"{PATH}/poses_ba", "*.txt"))
    poses_list = sort_poses(raw_poses_list)
    intrinsics = f"{PATH}/intrinsics.txt"
    bbs = read_list_box(box_list)
    Ms_t = read_list_poses(poses_list)
    visibility = np.ones((bbs.shape[0], 1))
    with open(intrinsics) as f:
        intr = f.readlines()
        K = np.array(
            [
                [float(intr[0]), 0, float(intr[2])],
                [0, float(intr[1]), float(intr[3])],
                [0, 0, 1],
            ]
        )
else:
    bbs = np.load('data/{:s}/bounding_boxes.npy'.format(dataset))  
    K = np.load('data/{:s}/intrinsics.npy'.format(dataset))
    Ms_t = np.load('data/{:s}/camera_poses.npy'.format(dataset)) 
    visibility = np.load('data/{:s}/visibility.npy'.format(dataset)) 

Compute the number of frames and the number of objects for the current dataset from the size of the visibility matrix.


In [32]:
n_frames, n_objects = visibility.shape
[inputCs, estCs, estQs] = compute_estimates(bbs, K, Ms_t, visibility)
Qs = estQs[0]

In [33]:
def dual_quadric_to_ellipsoid_parameters(Q):
    """Computes centre, axes length and orientation of one ellipsoid.

   A [4x4] matrix can represent general quadrics. In spite of preconditioning, the estimated quadrics can still,
   in rare cases, represent something other than ellipsoids. This is corrected by forcing the lengths of the
   axes to be positive.

    :param Q: Ellipsoid/Quadric in dual form [4x4].

    :returns:
      - centre - Ellipsoid centre in Cartesian coordinates [3x1].
      - axes - Length of ellipsoid axes [3x1].
      - R - Orientation of the ellipsoid [3x3].
    """

    # Scale the ellipsoid to put it in the usual form, with Q[3,3] = -1.
    Q = Q / (-Q[3, 3])

    # Compute ellipsoid centred on origin.
    centre = -Q[:3, 3]
    T = np.vstack((np.array((1, 0, 0, -centre[0])), np.array((0, 1, 0, -centre[1])),
                   np.array((0, 0, 1, -centre[2])), np.array((0, 0, 0, 1))))
    Qcent = T.dot(Q).dot(T.transpose())

    # Compute axes and orientation.
    [D, V] = np.linalg.eig(Qcent[:3, :3])
    sort_ev = np.argsort(D)
    V = np.vstack((V[:, sort_ev[0]], V[:, sort_ev[1]], V[:, sort_ev[2]])).transpose()
    D.sort()
    if sum(D < 0) > 0:
        # Take the absolute value of eigenvalues (they can be negative because of numerical issues).
        for index in range(D.shape[0]):
            if D[index] < 0:
                D[index] *= -1

    a = np.sqrt(D[0])
    b = np.sqrt(D[1])
    c = np.sqrt(D[2])
    
    axes = np.array([a, b, c])
    R = V
    return centre, axes, R